# Bangla Cyberbullying Binary Classification with XGB

This notebook implements a xgb classifier for binary classification of Bangla cyberbullying text (bullying vs. non-bullying).

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_curve,
    auc,
    precision_recall_curve,
)
from sklearn.preprocessing import label_binarize
from sklearn.discriminant_analysis import StandardScaler
from bnlp import SentencepieceTokenizer
from bnlp.embedding.fasttext import BengaliFasttext
from sklearn.preprocessing import LabelEncoder
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings
import torch
import os
from transformers import AutoTokenizer, AutoModel
from xgboost import XGBClassifier

warnings.filterwarnings("ignore")

# Set plot style
plt.style.use("ggplot")
sns.set(font_scale=1.2)
sns.set_style("whitegrid")

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import time

In [ ]:
MY_DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.get_device_name(MY_DEVICE)

In [ ]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "caching_allocator"

## 1. Loading and Exploring Datasets

In [ ]:
# 1. Load the dataset
print("Loading dataset...")
original_cleaned_df = pd.read_csv("../../dataset/cleaned/original_cleaned.csv")

print(f"Original dataset shape: {original_cleaned_df.shape}")

In [ ]:
# Display the first few rows of original dataset
original_cleaned_df.head()

## 2. Label Mapping and Visualization

In [ ]:
# Map the labels for binary classification (0 for 'not bully', 1 for all other classes)
def binary_label_mapping(label):
    return 0 if label == "not bully" else 1

original_cleaned_df["label_encoded"] = original_cleaned_df["label"].apply(binary_label_mapping)

In [ ]:
# Check the distribution of binary labels
plt.figure(figsize=(10, 5))
sns.countplot(x="label_encoded", data=original_cleaned_df, palette="viridis")
plt.title("Distribution of Binary Labels in Original Dataset")
plt.xticks([0, 1], ["Not Bullying (0)", "Bullying (1)"])
plt.tight_layout()
plt.show()

## 3. Data Splitting

In [ ]:
# Split the original dataset into train and test sets
X = original_cleaned_df["comment"]
y = original_cleaned_df["label_encoded"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

In [ ]:
# Visualize the binary class distribution in training set
plt.figure(figsize=(10, 6))
sns.countplot(x=y_train, palette="viridis")
plt.title("Binary Class Distribution in Training Set")
plt.xlabel("Class Label")
plt.xticks([0, 1], ["Not Bullying (0)", "Bullying (1)"])
plt.ylabel("Count")
plt.tight_layout()
plt.show()

## 4. Text Tokenization and Vectorization Via BanglaBERT

In [ ]:
bangla_bert_tokenizer = AutoTokenizer.from_pretrained("sagorsarker/bangla-bert-base")
bangla_bert_model = AutoModel.from_pretrained("sagorsarker/bangla-bert-base")

In [ ]:
def fit_and_return_hidden_layer(input_text_array: np.ndarray):
    """
    Fit the input array to the Bengali BERT model and return the concatenated last 4 hidden layers.
    """
    torch.cuda.empty_cache()
    tokenized_input = bangla_bert_tokenizer(
        input_text_array.tolist(),
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=100,
    )

    # Move the tokenized input to GPU
    tokenized_input = {k: v.to(MY_DEVICE) for k, v in tokenized_input.items()}

    # Put model in eval mode and move to GPU if not already there
    bangla_bert_model.eval()
    bangla_bert_model.to(MY_DEVICE)

    # Process in batches to avoid memory issues
    batch_size = 32
    num_samples = tokenized_input["input_ids"].shape[0]
    all_hidden_layers = []

    with torch.no_grad():  # Don't compute gradients for inference
        for i in range(0, num_samples, batch_size):
            batch_input = {k: v[i : i + batch_size] for k, v in tokenized_input.items()}

            with torch.amp.autocast(device_type="cuda"):
                outputs = bangla_bert_model(**batch_input, output_hidden_states=True)
                hidden_states = (
                    outputs.hidden_states
                )  # Tuple containing all hidden layers

                # Get the last 4 hidden layers and concatenate them
                last_4_layers = torch.cat(hidden_states[-4:], dim=-1)

                # Average across all tokens (mean pooling)
                hidden_layer = last_4_layers.mean(dim=1)

            # Move to CPU and detach from computation graph
            hidden_layer_cpu = hidden_layer.detach().cpu()
            reshaped_hidden_layer = hidden_layer_cpu.reshape(
                hidden_layer_cpu.shape[0], -1
            )
            all_hidden_layers.append(reshaped_hidden_layer)

    # Concatenate all batches
    combined_hidden_layers = torch.cat(all_hidden_layers, dim=0)

    return combined_hidden_layers.numpy()

In [ ]:
# Apply tokenization and vectorization to training data
print("Vectorizing training data...")
start_time = time.time()
X_train_vectors = fit_and_return_hidden_layer(X_train.values)
print(
    f"Training data vectorization completed in {time.time() - start_time:.2f} seconds"
)
print(f"Training data shape: {X_train_vectors.shape}")

In [ ]:
# Apply tokenization and vectorization to testing data
print("Vectorizing testing data...")
start_time = time.time()
X_test_vectors = fit_and_return_hidden_layer(X_test.values)
print(f"Testing data vectorization completed in {time.time() - start_time:.2f} seconds")

In [ ]:
# For runing without PCA
X_train_pca = X_train_vectors
X_test_pca = X_test_vectors

# Dimensionality reduction to visualize the features


# scaler = StandardScaler()

# print("Performing dimensionality reduction...")
# start_time = time.time()

# # First use PCA to reduce to 50 dimensions (makes t-SNE faster)
# print("Applying PCA...")
# pca = PCA(n_components=0.95, random_state=42)  # Keep 95% of variance
# X_train_pca = pca.fit_transform(scaler.fit_transform(X_train_vectors))
# X_test_pca = pca.transform(scaler.transform(X_test_vectors))
# print(f"PCA completed in {time.time() - start_time:.2f} seconds")

# Then apply t-SNE on the PCA results
# print("Applying t-SNE...")
# tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_jobs=-1)
# X_train_tsne = tsne.fit_transform(X_train_pca)
# print(f"t-SNE completed in {time.time() - start_time:.2f} seconds")

# # Visualize the 2D projection
# plt.figure(figsize=(12, 10))
# scatter = plt.scatter(
#     X_train_tsne[:, 0],
#     X_train_tsne[:, 1],
#     c=y_train_augmented,
#     alpha=0.5,
#     cmap="viridis",
# )
# plt.colorbar(scatter, label="Class")
# plt.title("t-SNE Visualization of BERT Embeddings")
# plt.xlabel("t-SNE Dimension 1")
# plt.ylabel("t-SNE Dimension 2")
# plt.tight_layout()
# plt.show()

## 5. Training XGB Model

In [ ]:
# Configure and train XGB classifier
print("Training XGB model...")
# Parameters optimized for text embeddings
xgb_model = XGBClassifier(
    n_estimators=200,  # Number of boosting rounds
    max_depth=6,  # Maximum tree depth
    learning_rate=0.1,  # Step size shrinkage used to prevent overfitting
    min_child_weight=1,  # Minimum sum of instance weight needed in a child
    gamma=0,  # Minimum loss reduction required for further partition
    subsample=0.8,  # Fraction of samples used for fitting trees
    colsample_bytree=0.8,  # Fraction of features used for fitting trees
    scale_pos_weight=1,  # Control the balance of positive and negative weights
    objective="binary:logistic",  # Binary classification objective
    random_state=42,
    n_jobs=-1,  # Use all processors
)

start_time = time.time()
xgb_model.fit(X_train_pca, y_train)
training_time = time.time() - start_time
print(f"Model trained in {training_time:.2f} seconds")

## 6. Model Evaluation - Basic Metrics

In [ ]:
# Make predictions and calculate basic metrics
print("Evaluating model...")
y_pred = xgb_model.predict(X_test_pca)

# Calculate basic metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="weighted")
recall = recall_score(y_test, y_pred, average="weighted")
f1 = f1_score(y_test, y_pred, average="weighted")

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

In [ ]:
# Display classification report for binary classification
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=["Not Bullying", "Bullying"]))

## 7. Confusion Matrix Visualization

In [ ]:
# Create and display confusion matrix for binary classification
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred)
cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]  # Normalize
sns.heatmap(
    cm_norm,
    annot=True,
    fmt=".2f",
    cmap="Blues",
    xticklabels=["Not Bullying", "Bullying"],
    yticklabels=["Not Bullying", "Bullying"],
)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Binary Classification Confusion Matrix")
plt.tight_layout()
plt.show()

## 8. Feature Importance Analysis

In [ ]:
# Analyze feature importance
feature_importance = pd.DataFrame(
    {
        "Feature": [f"Feature_{i}" for i in range(X_train_pca.shape[1])],
        "Importance": xgb_model.feature_importances_,
    }
)
feature_importance = feature_importance.sort_values("Importance", ascending=False).head(
    20
)

plt.figure(figsize=(12, 8))
sns.barplot(x="Importance", y="Feature", data=feature_importance, palette="viridis")
plt.title("Top 20 Important Features")
plt.tight_layout()
plt.show()

## 9. ROC Curve Analysis

In [ ]:
# ROC Curve and AUC for binary classification
y_score = xgb_model.predict_proba(X_test_pca)[:, 1]  # Probability of positive class

# Calculate ROC curve and ROC area
fpr, tpr, _ = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, color="darkorange", lw=2, label=f"ROC curve (area = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Binary Classification ROC Curve")
plt.legend(loc="lower right")
plt.tight_layout()
plt.show()

## 10. Precision-Recall Curve Analysis

In [ ]:
# Calculate Precision-Recall curve for binary classification
precision, recall, _ = precision_recall_curve(y_test, y_score)
avg_precision = precision_score(y_test, y_pred)

plt.figure(figsize=(10, 8))
plt.plot(
    recall,
    precision,
    color="blue",
    lw=2,
    label=f"Precision-Recall curve (AP = {avg_precision:.2f})",
)
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title("Binary Classification Precision-Recall Curve")
plt.legend(loc="lower left")
plt.tight_layout()
plt.show()

## 12. Model Summary and Saving

In [ ]:
# Summary of the binary classification model
print("Binary Classification Model Summary:")
print(f"Total samples in training set (without augmentation): {len(X_train)}")
print(f"Total samples in test set: {len(X_test)}")
print(f"Number of features (vector dimension): {X_train_vectors.shape[1]}")
print(f"Final model accuracy on test set: {accuracy:.4f}")
plt.show()

In [ ]:
# Save the model if needed
import joblib

joblib.dump(xgb_model, "../../models/bangla_bert/no_augment_binary_bangla_bert_tuned_xgb.pkl")